<a href="https://colab.research.google.com/github/etherealxx/volatile-concentration-localux-colab/blob/main/volatile_concentration_localux_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##***Project `VCL-colab`***
### All camenduru colab in one spot, synced in realtime
###### Last time updated: Jul 20, 23
###### (something doesn't work properly? Make sure you use the [latest version](https://colab.research.google.com/github/etherealxx/volatile-concentration-localux-colab/blob/main/volatile_concentration_localux_colab.ipynb), or [report a bug](https://github.com/etherealxx/volatile-concentration-localux-colab/issues).)

######  
To use:
- Click the play button
- Wait for less than a minute, until a UI with **big orange button** shows up
- Choose the model you want, **wait** for the **extension checkboxes** to shows up, choose the extension that you wanted, then click the orange button
- Just wait for about 8 minute until the webui links shows up on the bottom

About the options:
- `repo_branch`: Read [here](https://github.com/camenduru/stable-diffusion-webui-colab) (just use `stable` as default)
- `choose_models`: Untick this if you're relaunching the UI, or if you don't want to choose model (The webui will always download sd 1.4 model instead)
- `controlnet_model`: Tick this if you want to download controlnet models (for controlnet usage). Untick this if you're relaunching the UI. Note that `lite` branch won't download any controlnet model regardless if this checkbox is being ticked or not.

Got an Error 403 instead of the UI with big orange button? Read [here](https://github.com/etherealxx/volatile-concentration-localux-colab/blob/main/error403guide.md) to fix it.

## ⠀
(open the hidden cell to show help☝️)

In [ ]:
#@title 👇 Run this
repo_branch = 'stable' #@param ["lite", "stable", "nightly"]
choose_models = True #@param {type:"boolean"}
controlnet_models = False #@param {type:"boolean"}

print("[1;32mWait for about a minute until a UI with a big orange button shows up below")
print('[0m')

from IPython.display import clear_output
import time
%cd /content

try:
  start_colab
except:
  start_colab = int(time.time())-5
# clear_output()

%env TF_CPP_MIN_LOG_LEVEL=1
!git clone https://github.com/etherealxx/volatile-concentration-localux-colab /content/vcltools
!pip install -q gradio==3.28.3

emptymodel = False

colaboptions = {
  "branch": repo_branch,
  "filename": "stable_diffusion_1_5_webui_colab.ipynb",
  "part": "part2",
  "controlnet": controlnet_models,
  "download_model": choose_models,
  "empty_model": emptymodel
} #default

import os, pickle, sys, re, shutil
import gradio as gr

vclvarpath = '/content/vclvariables'
os.makedirs(vclvarpath, exist_ok=True)
!python /content/vcltools/cloneall3.py
!python /content/vcltools/choosemodel4.py {repo_branch}

def pickledump(vartodump, outputfile):
  outputpath = os.path.join(vclvarpath, outputfile + '.pkl')
  with open(outputpath, 'wb') as f:
      pickle.dump(vartodump, f)

def pickleload(prevvalue, inputfile):
  inputpath = os.path.join(vclvarpath, inputfile + '.pkl')
  if os.path.exists(inputpath):
      with open(inputpath, 'rb') as f:
          vartopass = pickle.load(f)
          return vartopass
  else:
    return prevvalue

sortedcolabname = pickleload(None, 'sortedcolabname')

!pip install base58
import base58
tunneltext = base58.b58decode("FFSgE5cGo6ewBdZ8cwDovS".encode()).decode()

clear_output(wait=True)

everycolab = f'/content/camendurus/{repo_branch}'
modelischosen = False
modelchosenmessage = ''
chosencolabname = ''
extensionlines = []
extensiontoremove = []

def list_additional_ext():
  addext_txtpath = "/content/vcltools/additionalextensions.txt"
  with open(addext_txtpath, 'r') as file:
    lines = [line.rstrip('\n') for line in file]
    exts = [ext for ext in lines if ext != "" and not ext.startswith("#")]
    return exts

additionalextension_names = [x.split("/")[-1] for x in list_additional_ext()]

def modelchosen(chosenmodel, chosenextension):
  global modelischosen
  global modelchosenmessage
  global chosencolabname
  global colaboptions
  global choose_models
  global extensionlines
  global extensiontoremove
  global emptymodel
  usedextensionlinks = []
  extensionlist = []
  # for link in extensionlines:
  #     match = re.search(r"https://github.com/\S+/(\S+)", link)
  #     if match:
  #         repo_name = match.group(1)
  #         if repo_name in chosenextension:
  #             usedextensionlinks.append(link)
  # pickledump(usedextensionlinks, '/content/extensions.pkl')
  pickledump(chosenextension, 'chosenextensions')
  fullextension = pickleload(None, 'fullextensions')
  if fullextension:
    extensiontoremove = [x for x in fullextension if x not in chosenextension]
  pickledump(extensiontoremove, 'removedextensions')
  if chosenmodel:
    modelchosenmessage = "[1;32mModel " + chosenmodel + " was chosen. It'll be downloaded soon. Continue scroll down and wait for the webui to be loaded."
    textbox_text = f"Model '{chosenmodel} was chosen. It'll be downloaded soon. Continue scroll down and wait for the webui to be loaded."
    chosencolabname = chosenmodel
    emptymodel = False
  else:
    modelchosenmessage = "[1;32mNo model was chosen. Continue scroll down and wait for the webui to be loaded."
    textbox_text = f"No model was chosen. Continue scroll down and wait for the webui to be loaded."
    choose_models = False
    colaboptions["download_model"] = False
    emptymodel = True

    !python /content/vcltools/extensioninstaller3.py
    extensionlines = pickleload(extensionlines, 'extensions')

    for ext_gitcloneline in extensionlines:
        pattern = r"https://github.com/\S+/(\S+)"
        match = re.search(pattern, ext_gitcloneline)
        if match:
            ext_name = match.group(1)
            if not ext_name == tunneltext:
              extensionlist.append(ext_name)
            # print(repo_name)
    pickledump(extensionlist, 'fullextensions')

  modelischosen = True
  return [gr.Button.update(visible=False), gr.Textbox.update(value=textbox_text, visible=True), gr.CheckboxGroup.update(visible=False),  gr.Dropdown.update(interactive=False)]

def extensionschoose(modelthatischosen):
    extensionlist = []
    global colaboptions
    global extensionlines
    global additionalextension_names
    if modelthatischosen:
        colabfilename = modelthatischosen + '_webui_colab.ipynb'
        colaboptions["filename"] = colabfilename
        chosencolabfilepath = os.path.join(everycolab, colabfilename)
    pickledump(colaboptions, 'colaboptions')

    !python /content/vcltools/extensioninstaller3.py
    extensionlines = pickleload(extensionlines, 'extensions')

    for ext_gitcloneline in extensionlines:
        pattern = r"https://github.com/\S+/(\S+)"
        match = re.search(pattern, ext_gitcloneline)
        if match:
            ext_name = match.group(1)
            if not ext_name == tunneltext:
              extensionlist.append(ext_name)
            # print(repo_name)
    pickledump(extensionlist, 'fullextensions')
    # extensionlist.sort()
    !python /content/vcltools/additionalextension.py
    templist = pickleload(None, 'tempext')
    chosenextension = pickleload(templist, 'chosenextensions')

    extensionlist = list(set(extensionlist) - set(additionalextension_names))
    extensionlist.sort()
    additionalextension_names.sort()
    # extensionlist.extend(chosenextension)
    for extname in additionalextension_names:
      extensionlist.append(extname)

    return gr.CheckboxGroup.update(choices=extensionlist, value=chosenextension, visible=True)

def dumpchoice(extensionthatischosen):
  pickledump(extensionthatischosen, 'chosenextensions')

with gr.Blocks() as vclcolab:
    with gr.Column():
      choose = gr.Dropdown(sortedcolabname, label="Choose Your Model", interactive=True)
      confirm = gr.Button("Use This Model", variant="primary", visible=True)
      donetext = gr.Textbox("", label="", visible=False)
      extchooser = gr.CheckboxGroup(["placeholder"], label="Extensions", info="Untick the checkboxes on those extensions that you want to disable", visible=False, interactive=True)
      confirm.click(modelchosen, inputs=[choose, extchooser], outputs=[confirm, donetext, extchooser, choose])
      choose.change(extensionschoose, inputs=[choose], outputs=[extchooser])
      extchooser.change(dumpchoice, inputs=[extchooser])

gr.close_all()
if choose_models:
  tryport = 7860
  while True:
    tryport += 1
    try:
      print("[1;32mNo need to click on the public url yet. Just wait until a UI with a big orange button shows up below")
      print("[1;32mIf 'Error 403' shows instead, click the stop button twice until 'Gradio UI interrupted' message shows up, then read here: https://github.com/etherealxx/volatile-concentration-localux-colab/blob/main/error403guide.md")
      print('[0m')
      vclcolab.launch(server_port=tryport)
      break
    except OSError:
      pass

  while True:
    try:
      if modelischosen:
        clear_output(wait=True)
        print(modelchosenmessage)
        print('[0m')
        vclcolab.close()
        break
    except KeyboardInterrupt:
      clear_output(wait=True)
      print("[1;33mGradio UI interrupted")
      print('[0m')
      vclcolab.close()
      # sys.exit()
      # os._exit(1)
      break

aria2c_lines = []
colabfilename = ''
chosencolabfilepath = os.path.join(everycolab, colaboptions["filename"])

if chosencolabname:
   colabfilename = chosencolabname + '_webui_colab.ipynb'
   colaboptions["filename"] = colabfilename
   chosencolabfilepath = os.path.join(everycolab, colabfilename)

if os.path.exists(chosencolabfilepath):
  print("[1;32mGathering aria2c code from " + chosencolabfilepath + "...")
  print('[0m')
  with open(chosencolabfilepath, 'r', encoding='utf-8') as f:
      for line in f:
          stripped_line = line.strip()
          if stripped_line.startswith('"!aria2c'):
              aria2c_lines.append(stripped_line)

if aria2c_lines:
  pickledump(aria2c_lines, 'arialist')

colaboptions["empty_model"] = emptymodel

pickledump(colaboptions, 'colaboptions')

# !python /content/vcltools/camendurucolab.py {colabfilename} {repo_branch} part1
!apt -y update -qq
!python /content/vcltools/camendurucolab3.py #{colabfilename} {repo_branch} part2
!python /content/vcltools/runaria3.py
%cd /content/volatile-concentration-localux

# if extensiontoremove and not emptymodel:
#   for ext in extensiontoremove:
#     extpath = os.path.join('/content/volatile-concentration-localux/extensions', ext)
#     if os.path.exists(extpath):
#       shutil.rmtree(extpath)
#       print(f"removed {ext} extension")

# !python /content/vcltools/camendurucolab.py {colabfilename} {repo_branch} part3
!sed -i -e '''/from modules import launch_utils/a\import os''' /content/volatile-concentration-localux/launch.py
!sed -i -e '''/        prepare_environment()/a\        os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' /content/volatile-concentration-localux/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' /content/volatile-concentration-localux/launch.py
!sed -i -e 's/\"sd_model_checkpoint\"\]/\"sd_model_checkpoint\"\,\"sd_vae\"\,\"CLIP_stop_at_last_layers\"\]/g' /content/volatile-concentration-localux/modules/shared.py

print("[1;32mlaunching the webui...")
print('[0m')

!python /content/vcltools/setuptimer.py
!echo -n {start_colab} > /content/volatile-concentration-localux/static/colabTimer.txt
!wget -q https://raw.githubusercontent.com/etherealxx/batchlinks-webui/main/notification.mp3 -P /content/volatile-concentration-localux

!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

(If you got 'Error 403' when the big orange button is supposed to shows up, read [here](https://github.com/etherealxx/volatile-concentration-localux-colab/blob/main/error403guide.md) for the fix)

## ⠀

 ### (debug area)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
!git clone -b dev5 https://github.com/etherealxx/volatile-concentration-localux-colab /content/vcltools

In [ ]:
import os
import shutil

def copy_files(source_dir, destination_dir):
    # Create destination directory if it doesn't exist
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)
        print(f"Created directory: {destination_dir}")

    # Get the list of files in the source directory
    files = os.listdir(source_dir)

    for file in files:
        source_file = os.path.join(source_dir, file)
        destination_file = os.path.join(destination_dir, file)

        # Copy the file to the destination directory
        shutil.copy2(source_file, destination_file)

        # Print success message with the filename
        print(f"Successfully copied {file}.")

# Specify the source and destination directories
folder_A = "/content/volatile-concentration-localux/outputs/txt2img-images/xxx"
folder_B = "/content/drive/MyDrive/xxx"

# Copy files from folder A to folder B
copy_files(folder_A, folder_B)

In [ ]:
%cd /content
!git clone https://github.com/etherealxx/volatile-concentration-localux-colab /content/colabtools

In [ ]:
!rm -rf /content/volatile-concentration-localux
!rm -rf /content/camendurus
!rm /content/frpc_linux_amd64

In [ ]:
import os, pickle

toopen = '/content/vclvariables/extensions.pkl'

if os.path.exists(toopen):
  with open(toopen, 'rb') as f:
      arialines2 = pickle.load(f)

for line in arialines2:
  print(line)

In [ ]:
repo_branch = 'nightly' #@param ["lite", "stable", "nightly"]

colaboptions = {
  "branch": repo_branch,
  "filename": "stable_diffusion_1_5_webui_colab.ipynb",
  "part": "part2"
} #default


print(colaboptions['branch'])

In [ ]:
with open('/content/camendurus/nightly/a_certain_model_webui_colab.ipynb', 'r', encoding='utf-8') as f:
    for line in f:
        stripped_line = line.strip()
        if stripped_line.startswith('"!aria2c'):
            # aria2c_lines.append(stripped_line)
            print("[1;32m" + stripped_line)
            print('[0m')
        else:
          print(stripped_line)

In [ ]:
checktest = None
with open('/content/arialist.pkl', 'rb') as f:
  checktest = pickle.load(f)
print(checktest)